In [1]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [2]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [3]:
session = boto3.Session(profile_name='master')

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [7]:
path = "s3://kosmobiker-masterproject/data/raw/"

In [8]:
import awswrangler as wr


In [9]:
s3 = session.client('s3')

In [10]:
s3_clientobj = s3.get_object(Bucket='kosmobiker-masterproject', Key='data/raw/ape_in_records_historical_prices.json')

In [11]:
d = s3_clientobj['Body'].read().decode('utf-8')

In [12]:
json_data = json.loads(d)

In [13]:
def get_json_from_s3(bucket, key, session):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [91]:
res_empty = get_json_from_s3("kosmobiker-masterproject", 'data/raw/ape_in_records_historical_prices.json', session)
res_bit = get_json_from_s3("kosmobiker-masterproject", 'data/raw/bitcoin_historical_prices.json', session)
res_amar = get_json_from_s3("kosmobiker-masterproject", 'data/raw/amateras_historical_prices.json', session)
res_aaa = get_json_from_s3("kosmobiker-masterproject", 'data/raw/american-shiba_historical_prices.json', session)

In [61]:
def make_df(json_data):
    return pd.DataFrame({
                "timestamp" : [i[0] for i in json_data['prices']],
                "prices" : [i[1] for i in json_data['prices']],
                "market_caps" : [i[1] for i in json_data['market_caps']],
                "total_volumes" : [i[1] for i in json_data['total_volumes']]
    })
    


In [92]:
make_df(res_aaa).dtypes

timestamp          int64
prices           float64
market_caps      float64
total_volumes    float64
dtype: object

In [94]:
make_df(res_aaa)

,timestamp,prices,market_caps,total_volumes
0,1621468800000,2.045221e-10,0.0,1.915088e+06
1,1621555200000,2.045221e-10,0.0,1.915088e+06
2,1621641600000,1.581390e-10,0.0,1.057736e+06
3,1621728000000,1.643583e-10,0.0,5.306828e+05
4,1621814400000,1.226077e-10,0.0,3.308410e+05
...,...,...,...,...
301,1647475200000,4.239951e-11,0.0,1.256508e+03
302,1647561600000,4.289962e-11,0.0,5.387097e+02
303,1647648000000,4.479691e-11,0.0,1.309825e+02
304,1647734400000,4.401541e-11,0.0,3.012487e+03


In [45]:
from jsonschema import validate
import json


In [54]:

with open('/home/vlad/master_project/config/schema.json') as json_file:
    schema = json.load(json_file)


In [62]:
validate(instance=res_bit, schema=schema)

ValidationError: 8290260777.375 is not of type 'integer'

Failed validating 'type' in schema['properties']['market_caps']['items']['items']:
    {'$id': '#root/market_caps/items/items',
     'default': 10,
     'maxItems': 2,
     'minimum': 0,
     'title': 'Items',
     'type': 'integer'}

On instance['market_caps'][406][1]:
    8290260777.375

In [83]:
def validate_json(data, schema):
    try:
        validate(instance=data, schema=schema)
        if len(data['prices']) < 30:
            raise ValueError
        return True
    except Exception:
        return False



In [85]:
validate_json(res_empty, schema)

False